# Subject Recognition CRF Tagger Model

In [1]:
import sys
sys.path.insert(0, '../../../allennlp')
sys.path.insert(0, '../../')

In [2]:
import os
from allennlp.common.params import Params

crf_tagger = {
    "dataset_reader": {
        "type": "sequence_tagging",
        "word_tag_delimiter": "/",
        "token_indexers": {
            "tokens": {
                "type": "single_id",
                "lowercase_tokens": True
            },
            "token_characters": {
                "type": "characters",
                "character_tokenizer": {
                    "end_tokens": [
                        "@@PADDING@@",
                        "@@PADDING@@",
                        "@@PADDING@@",
                        "@@PADDING@@"
                    ]
                }
            }
        }
    },
    # TODO: Update this to the location for subject name recognition training data
    "train_data_path": './../../data/subject_recognition/train.txt', 
    "validation_data_path": './../../data/subject_recognition/dev.txt',
    "model": {
        "type": "crf_tagger",
        "text_field_embedder": {
            "tokens": {
                "type": "embedding",
                "pretrained_file": "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.100d.txt.gz",
                "embedding_dim": 100,
                "trainable": False
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 16
                },
                "encoder": {
                    "type": "cnn",
                    "embedding_dim": 16,
                    "num_filters": 100,
                    "ngram_filter_sizes": [
                        5
                    ]
                },
                "dropout": 0.6
            }
        },
        "encoder": {
            "type": "lstm",
            "input_size": 200,
            "hidden_size": 300,
            "num_layers": 3,
            "dropout": 0.2,
            "bidirectional": True
        }
    },
    "iterator": {
        "type": "bucket",
        "sorting_keys": [
            [
                "tokens",
                "num_tokens"
            ]
        ],
        "batch_size": 16
    },
    "trainer": {
        "optimizer": "adam",
        "num_epochs": 10,
        "grad_norm": 1.0,
        "patience": 3,
        "cuda_device": 0
    }
}

In [3]:
import logging
# Create root logger
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

from allennlp.commands.train import train_model
from lib.utils import get_log_directory_path

params = Params(crf_tagger)

serialization_dir = get_log_directory_path('subject_recognition_crf_tagger', './../../logs')
print('Serialization Directory:', serialization_dir)
train_model(params=params, serialization_dir=serialization_dir)

Serialization Directory: ./../../logs/0000.01-15_21:34:34.subject_recognition_crf_tagger
2018-01-15 21:34:34,010 | INFO : random_seed = 13370
2018-01-15 21:34:34,011 | INFO : numpy_seed = 1337
2018-01-15 21:34:34,011 | INFO : pytorch_seed = 133
2018-01-15 21:34:34,013 | INFO : Pytorch version: 0.3.0.post4
2018-01-15 21:34:34,015 | INFO : dataset_reader.type = sequence_tagging
2018-01-15 21:34:34,016 | INFO : dataset_reader.token_indexers.tokens.type = single_id
2018-01-15 21:34:34,016 | INFO : dataset_reader.token_indexers.tokens.namespace = tokens
2018-01-15 21:34:34,016 | INFO : dataset_reader.token_indexers.tokens.lowercase_tokens = True
2018-01-15 21:34:34,017 | INFO : dataset_reader.token_indexers.token_characters.type = characters
2018-01-15 21:34:34,017 | INFO : dataset_reader.token_indexers.token_characters.namespace = token_characters
2018-01-15 21:34:34,018 | INFO : dataset_reader.token_indexers.token_characters.character_tokenizer.byte_encoding = None
2018-01-15 21:34:34,018

74520it [00:02, 33110.60it/s]


2018-01-15 21:34:36,338 | INFO : validation_data_path = ./../../data/subject_recognition/dev.txt
2018-01-15 21:34:36,339 | INFO : Reading validation data from ./../../data/subject_recognition/dev.txt
2018-01-15 21:34:36,339 | INFO : Reading instances from lines in file at: ./../../data/subject_recognition/dev.txt


10648it [00:00, 20577.30it/s]

2018-01-15 21:34:36,868 | INFO : test_data_path = None
2018-01-15 21:34:36,869 | INFO : Creating a vocabulary using validation, train data.


...

token_accuracy: 0.98, accuracy: 0.97, loss: 2.21 ||: 100%|##########| 4658/4658 [01:53<00:00, 41.04it/s]


2018-01-15 21:47:20,568 | INFO : Validating


token_accuracy: 0.98, accuracy: 0.95, loss: 5.90 ||: 100%|##########| 666/666 [00:08<00:00, 74.60it/s]

2018-01-15 21:47:29,499 | INFO : Ran out of patience.  Stopping training.
2018-01-15 21:47:29,500 | INFO : archiving weights and vocabulary to ./../../logs/0000.01-15_21:34:34.subject_recognition_crf_tagger/model.tar.gz


CrfTagger(
  (text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding(
    )
    (token_embedder_token_characters): TokenCharactersEncoder(
      (_embedding): TimeDistributed(
        (_module): Embedding(
        )
      )
      (_encoder): TimeDistributed(
        (_module): CnnEncoder(
          (_activation): ReLU()
          (conv_layer_0): Conv1d (16, 100, kernel_size=(5,), stride=(1,))
        )
      )
      (_dropout): Dropout(p=0.6)
    )
  )
  (encoder): PytorchSeq2SeqWrapper(
    (_module): LSTM(200, 300, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (tag_projection_layer): TimeDistributed(
    (_module): Linear(in_features=600, out_features=2)
  )
  (crf): ConditionalRandomField(
  )
)